In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal as scipy_signal
from scipy.interpolate import PchipInterpolator
import neurokit2 as nk

In [ ]:
fs_temp = 4  

for participant_id in participants:

    filtered_folder = os.path.join(DATA_FINAL_DIR, participant_id, "Filtered_Signals")
    os.makedirs(filtered_folder, exist_ok=True)

    for side in ["LEFT", "RIGHT"]:

        # Load temperature
        temp_path = os.path.join(
            DATA_FINAL_DIR,
            participant_id,
            "Signals",
            f"{participant_id}_{side}_temp.csv",
        )


        df_temp = pd.read_csv(temp_path)
        temp_raw = df_temp["temperature"].values.copy()

        artifact_mask = np.zeros(len(temp_raw), dtype=bool)

        artifact_mask |= temp_raw < 25
        artifact_mask |= temp_raw > 40

        temp_diff = np.abs(np.diff(temp_raw, prepend=temp_raw[0]))
        artifact_mask |= temp_diff > 2.0

        artifact_pct = artifact_mask.sum() / len(temp_raw) * 100
        print(f"       Artifacts: {artifact_mask.sum():,} ({artifact_pct:.1f}%)")


        if artifact_pct > 90:
            temp_interpolated = temp_raw.copy()
            artifact_mask = np.zeros(len(temp_raw), dtype=bool)
        else:
            temp_masked = temp_raw.copy().astype(float)
            temp_masked[artifact_mask] = np.nan

            valid_idx = np.where(~np.isnan(temp_masked))[0]
            valid_vals = temp_masked[valid_idx]

            if len(valid_idx) > 2:
                pchip = PchipInterpolator(valid_idx, valid_vals, extrapolate=True)
                temp_interpolated = pchip(np.arange(len(temp_masked)))
            else:
                temp_interpolated = (
                    pd.Series(temp_masked).interpolate().ffill().bfill().values
                )


        cutoff_hz = 0.1
        order = 2
        nyquist = fs_temp / 2
        b, a = scipy_signal.butter(order, cutoff_hz / nyquist, btype="low")
        temp_filtered = scipy_signal.filtfilt(b, a, temp_interpolated)

        df_out = df_temp.copy()
        df_out["temp_raw"] = temp_raw
        df_out["temp_filtered"] = temp_filtered
        df_out["artifact"] = artifact_mask.astype(int)

        if "temperature" in df_out.columns:
            df_out = df_out.drop(columns=["temperature"])

        cols = [
            "participant_id",
            "side",
            "phase_start",
            "phase_end",
            "timestamp",
            "temp_raw",
            "temp_filtered",
            "artifact",
            "phase",
        ]
        cols = [c for c in cols if c in df_out.columns]
        df_out = df_out[cols]

        output_path = os.path.join(
            filtered_folder, f"{participant_id}_{side}_temp_filtered.csv"
        )
        df_out.to_csv(output_path, index=False)
        print(f"       Saved: {os.path.basename(output_path)}")